This is another project I did for fun: I built a nested for loop that will generate the probability matrix associated with the NBA draft lottery. The goal is to produce the same chart that is displayed in this wikipedia article (Scroll down to 2019) https://en.wikipedia.org/wiki/NBA_draft_lottery. It's a tough conditional probability problem to tackle. NOTE: Some of the for loops get cut off when converting this notebook to a PDF, so it might be difficult to see the full logic

In [ ]:
import numpy as np
import pandas as pd

# The following webpage helped me understand the math behind the probabilities
# It also helped me understand a programming framework for doing all of these calculations, although my code is original
# https://squared2020.com/2017/09/30/how-nba-draft-lottery-probabilities-are-constructed/

The following for loop does what is mentioned in the previous section

In [81]:
initialprobs = [0.14, 0.14, 0.14, 0.125, 0.105, 0.09, 0.075, 0.06, 0.045, 0.03, 0.02, 0.015, 0.01, 0.005]

second_round = [0]*14
third_round = [0]*14
fourth_round = [0]*14
s = (14, 10)
final_matrix = np.zeros(s)
for i in range(len(initialprobs)):
    pi = initialprobs[i]
    temp = initialprobs.copy()
    temp[i] = 0    
    for j in range(len(initialprobs)):
        if j!=i:
            pj = initialprobs[j]
            temp2 = temp.copy()
            temp2[j] = 0
            pi_frac = pi/(1 - pi)
            val = pi_frac*pj
            second_round[j] = second_round[j] + val
            for k in range(len(initialprobs)):
                if (k!=i) and (k!=j):
                    pk = initialprobs[k]
                    temp3 = temp2.copy()
                    temp3[k] = 0
                    pj_frac = pj/(1 - pi - pj)
                    val = pi_frac*pj_frac*pk
                    third_round[k] = third_round[k] + val
                    for l in range(len(initialprobs)):
                        if (l!=i) and (l!=j) and (l!=k):
                            pl = initialprobs[l]
                            temp4 = temp3.copy()
                            temp4[l] = 0
                            pk_frac = pk/(1 - pi - pj - pk)
                            val = pi_frac*pj_frac*pk_frac*pl
                            fourth_round[l] = fourth_round[l] + val
                            round_counter = 0
                            while sum(temp4)>0:
                                team_index = temp4.index(max(temp4))
                                final_matrix[team_index][round_counter] = final_matrix[team_index][round_counter] + val
                                temp4[team_index] = 0
                                round_counter += 1

The following code block repeats the same process above, but we change the number of rounds, and we change the odds. So the problem becomes noticeably different from the original. This is cool because you can change the odds and test out different outcomes without having to make crazy changes to the code.

In [92]:
initialprobs = [0.114, 0.113, 0.112, 0.111, 0.099, 0.089, 0.079, 0.069, 0.059, 0.049, 0.039, 0.029, 0.019, 0.009, 0.006, 0.004]

second_round = [0]*16 # initiate four lists full of zeros
third_round = [0]*16 # these zeros will get filled as we loop through each combination of teams selected by the lottery
fourth_round = [0]*16
fifth_round = [0]*16
s = (16, 11)
final_matrix = np.zeros(s) # initiate matrix full of zeros, this is for rounds 6-16 after the probabilistic lottery portion

# here, i is the first team selected in the lottery. We loop through all possible teams getting selected first
for i in range(len(initialprobs)): 
    pi = initialprobs[i]
    temp = initialprobs.copy()
    temp[i] = 0 # because team i is getting picked first, we take their probability out of the list by making it zero
    for j in range(len(initialprobs)): # then we loop through all teams getting picked second (j)
        if j!=i:
            pj = initialprobs[j]
            temp2 = temp.copy()
            temp2[j] = 0 # team j gets picked second, we take them out of the list
            pi_frac = pi/(1 - pi) 
            val = pi_frac*pj # pj*pi/(1 - pi) is the conditional probability that team j gets selected second after team i
            second_round[j] = second_round[j] + val # summing this up for all teams i gives us probability of j getting picked second
            for k in range(len(initialprobs)):
                if (k!=i) and (k!=j):
                    pk = initialprobs[k]
                    temp3 = temp2.copy()
                    temp3[k] = 0
                    pj_frac = pj/(1 - pi - pj) # as teams get selected, their probabilities are subtracted from the denominator
                    val = pi_frac*pj_frac*pk # multiplying these fractions together and then by pk gives us the conditional probability
                    third_round[k] = third_round[k] + val # in other words, probability team k gets picked third after i and then j
                    for l in range(len(initialprobs)):
                        if (l!=i) and (l!=j) and (l!=k):
                            pl = initialprobs[l]
                            temp4 = temp3.copy()
                            temp4[l] = 0
                            pk_frac = pk/(1 - pi - pj - pk)
                            val = pi_frac*pj_frac*pk_frac*pl # probability of team l getting picked after teams i, j, k
                            fourth_round[l] = fourth_round[l] + val # sum these conditional probabilities for all possible combinations
                            for m in range(len(initialprobs)):
                                if (m!=i) and (m!=j) and (m!=k) and (m!=l):
                                    pm = initialprobs[m]
                                    temp5 = temp4.copy()
                                    temp5[m] = 0
                                    pl_frac = pl/(1 - pi - pj - pk - pl)
                                    val = pi_frac*pj_frac*pk_frac*pl_frac*pm # probability of team m getting picked after teams i, j, k, l
                                    fifth_round[m] = fifth_round[m] + val
                                    
                                    # after the fifth round, no more probability is needed, the rest of the order is determined by seed
                                    # so, we can use the same P(m=5 | i=1,j=2,k=3,l=4) to determine the rest of the round odds
                                    round_counter = 0 # this will be used to count the rounds after the 5th round (6th round = 0)
                                    while sum(temp5)>0: # when the sum=0, this means all teams have been given a lottery position in this loop
                                        team_index = temp5.index(max(temp5)) # we use max to search for lowest seed remaining
                                        # here we fill the matrix of zeros with the conditional probability commented above
                                        final_matrix[team_index][round_counter] = final_matrix[team_index][round_counter] + val
                                        temp5[team_index] = 0 # making the team 0 is kind of equivalent to saying "this team finally got its pick"
                                        round_counter += 1 # after a team gets their pick, we move to next round of the lottery

In [99]:
# make sure our output looks appropriate
final_rounds = pd.DataFrame(final_matrix)
final_rounds

,0,1,2,3,4,5,6,7,8,9,10
0,0.469749,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
1,0.273960,0.198981,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
2,0.147744,0.254418,0.073992,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
3,0.071306,0.229152,0.155605,0.023330,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
4,0.029955,0.178902,0.225853,0.078715,0.006641,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
5,0.007286,0.103400,0.247828,0.163778,0.032707,0.001629,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
6,0.000000,0.035148,0.200783,0.252457,0.095718,0.011286,0.000330,0.000000,0.000000,0.000000e+00,0.000000e+00
7,0.000000,0.000000,0.095940,0.286552,0.205871,0.045860,0.003156,0.000052,0.000000,0.000000e+00,0.000000e+00
8,0.000000,0.000000,0.000000,0.195169,0.330166,0.138036,0.017764,0.000683,0.000006,0.000000e+00,0.000000e+00
9,0.000000,0.000000,0.000000,0.000000,0.328897,0.318826,0.075763,0.005354,0.000107,4.361609e-07,0.000000e+00


In [111]:
# various fixes to make the dataframe look nicer
first_5 = pd.DataFrame({'1st':initialprobs, '2nd':second_round, '3rd':third_round, '4th':fourth_round, '5th':fifth_round})
final_df = pd.concat([first_5, final_rounds], axis = 1)
final_df['Chances'] = final_df['1st']*1000
final_df['Seed'] = final_df.index+1
final_df = final_df.rename({0:'6th', 1:'7th', 2:'8th', 3:'9th', 4:'10th', 5:'11th', 6:'12th', 7:'13th', 8:'14th', 9:'15th', 10:'16th'}, axis = 1)
cols = final_df.columns.tolist()
cols = cols[-1:] + cols[-2:-1] + cols[:-2]
final_df = final_df[cols]
final_df # looks good!

,Seed,Chances,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th,11th,12th,13th,14th,15th,16th
0,1,114.0,0.114,0.110468,0.106537,0.102124,0.097122,0.469749,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
1,2,113.0,0.113,0.109643,0.105893,0.101667,0.096856,0.273960,0.198981,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
2,3,112.0,0.112,0.108815,0.105244,0.101203,0.096583,0.147744,0.254418,0.073992,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
3,4,111.0,0.111,0.107984,0.104589,0.100732,0.096302,0.071306,0.229152,0.155605,0.023330,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
4,5,99.0,0.099,0.097793,0.096328,0.094528,0.092286,0.029955,0.178902,0.225853,0.078715,0.006641,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
5,6,89.0,0.089,0.088999,0.088867,0.088548,0.087957,0.007286,0.103400,0.247828,0.163778,0.032707,0.001629,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
6,7,79.0,0.079,0.079941,0.080884,0.081803,0.082650,0.000000,0.035148,0.200783,0.252457,0.095718,0.011286,0.000330,0.000000,0.000000,0.000000e+00,0.000000e+00
7,8,69.0,0.069,0.070627,0.072380,0.074269,0.076293,0.000000,0.000000,0.095940,0.286552,0.205871,0.045860,0.003156,0.000052,0.000000,0.000000e+00,0.000000e+00
8,9,59.0,0.059,0.061064,0.063360,0.065930,0.068822,0.000000,0.000000,0.000000,0.195169,0.330166,0.138036,0.017764,0.000683,0.000006,0.000000e+00,0.000000e+00
9,10,49.0,0.049,0.051262,0.053830,0.056774,0.060187,0.000000,0.000000,0.000000,0.000000,0.328897,0.318826,0.075763,0.005354,0.000107,4.361609e-07,0.000000e+00


In [112]:
# save as csv
final_df.to_csv('DraftProbabilityTable.csv')